In [1]:
import cv2
import numpy as np

# --- ArUco dictionary (DICT_4X4_50) ---
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters() 

# --- Open the webcam ---
cap = cv2.VideoCapture(0)   # /dev/video0

if not cap.isOpened():
    print("Could not open webcam")
    exit()

while True:
    ret, frame = cap.read() # fram is a numpy array of shape (height, width, 3) of type uint8 -> 0 to 255
    if not ret:
        print("Frame not received")
        break
    # Convert to grayscale (Aruco works on grayscale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # gray is an array of shape (heigh, width, 1) type uint8
    # Detect markers
    corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params) 
   
    # Draw a square using corners: bottom_right id0, bottom_left id1 top_left id2
    if ids is not None:
        ids_flat = ids.flatten()
        pts = {}

        for idx, marker_id in enumerate(ids_flat):
            c = corners[idx][0]  # shape (4, 2): [tl, tr, br, bl]
            if marker_id == 0:
                # id0 at top-left of the map → inner corner = bottom-right
                pts[0] = c[2]  # br
            elif marker_id == 1:
                # id1 at top-right of the map → inner corner = bottom-left
                pts[1] = c[3]  # bl
            elif marker_id == 2:
                # id2 at bottom-right of the map → inner corner = top-left
                pts[2] = c[0]  # tl

        # Draw boundary by connecting inner corners if all 3 are visible
        if all(k in pts for k in (0, 1, 2)):
            p0 = pts[0]
            p1 = pts[1]
            p2 = pts[2]

            poly = np.array([p0, p1, p2], dtype=np.int32).reshape((-1, 1, 2))
            cv2.polylines(frame, [poly], isClosed=True, color=(0, 255, 0), thickness=2)

    # Show the frame
    cv2.imshow("Aruco Feed", frame)

    # Quit with q 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
cap.release()
cv2.destroyAllWindows()


In [5]:
import cv2
import numpy as np
import math

# Compute center of the ArUco marker 
def compute_center(top_left,bottom_right):
    center_x = int((top_left[0] + bottom_right[0]) / 2.0)
    center_y = int((top_left[1] + bottom_right[1]) / 2.0)
    return center_x,center_y
    
# Compute angle of vector = (bottom_left -> top_left) w.r.t horizontal axis 
def compute_angle(top_left,bottom_left):
    x_diff = int(top_left[0] - bottom_left[0]) 
    y_diff = int(top_left[1] - bottom_left[1]) 
    return math.atan2(y_diff,x_diff)

def detect_red_polygons(frame,
                        min_area=300,       # filter tiny blobs
                        eps_factor=0.02):   # approximation factor for vertices
    """
    Returns:
      polygons: list of (N,2) int arrays with polygon vertices in image coords
      mask: binary mask used to detect red regions
    """
    # BGR -> HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Red wraps around 0°, so use two ranges
    lower_red1 = np.array([0,   80,  80])
    upper_red1 = np.array([10,  255, 255])
    lower_red2 = np.array([170, 80,  80])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask  = cv2.bitwise_or(mask1, mask2)

    # Cleanup (optional but usually helpful)
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  kernel, iterations=2)

    # Contours --> polygons
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < min_area:
            continue

        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, eps_factor * peri, True)  # (N,1,2)
        verts = approx.reshape(-1, 2)                            # (N,2)
        polygons.append(verts.astype(int))

    return polygons, mask

def detect_red_polygons_in_map(frame, p_tl, p_br,min_area=300, eps_factor=0.02):
    x0, y0 = p_tl
    x1, y1 = p_br

    # Ensure proper ordering
    x0, x1 = sorted((x0, x1))
    y0, y1 = sorted((y0, y1))

    roi = frame[y0:y1, x0:x1]

    polygons_roi, mask_roi = detect_red_polygons(roi,min_area=min_area,eps_factor=eps_factor)
    # Shift vertices back to full-frame coordinates
    polygons_full = []
    offset = np.array([[x0, y0]])
    for verts in polygons_roi:
        polygons_full.append(verts + offset)
    
    return polygons_full, mask_roi

In [6]:
import cv2
import numpy as np

# --- ArUco dictionary (DICT_4X4_50) ---
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters() 

# --- Open the webcam ---
cap = cv2.VideoCapture(0)   # /dev/video0

if not cap.isOpened():
    print("Could not open webcam")
    exit()

while True:
    ret, frame = cap.read() # fram is a numpy array of shape (height, width, 3) of type uint8 -> 0 to 255
    if not ret:
        print("Frame not received")
        break
    # Convert to grayscale (Aruco works on grayscale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # gray is an array of shape (heigh, width, 1) type uint8
    # Detect markers
    corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params) 
    
    marker0_TL = marker1_BR = marker2_thymio = marker3_goal = 0
    
    if ids is not None and len(ids) >= 2:
        ids_flat = ids.flatten()
        
        for i, marker_id in enumerate(ids_flat):
            if marker_id == 0:
                marker0_TL = i
            elif marker_id == 1:
                marker1_BR = i
            elif marker_id == 2:
                marker2_thymio = i
            elif marker_id == 3:
                marker3_goal = i
        
        x0, y0 = corners[marker0_TL][0][2] # TL corner of map (2 = BR corner of marker0)
        x1, y1 = corners[marker1_BR][0][0] # BR corner of map (0 = TL corner of mareker1)
        
        # Four rectangle corners
        p_tl = (int(x0), int(y0))  
        p_br = (int(x1), int(y1))  
        p_tr = (int(x1), int(y0))
        p_bl = (int(x0), int(y1))

        cv2.line(frame, p_tl, p_tr, (0, 255, 0), 2)
        cv2.line(frame, p_tr, p_br,  (0, 255, 0), 2)
        cv2.line(frame, p_br,  p_bl, (0, 255, 0), 2)
        cv2.line(frame, p_bl, p_tl,  (0, 255, 0), 2)
        
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        thymio_x,thymio_y = compute_center(corners[marker2_thymio][0][0],corners[marker2_thymio][0][2]) # TL,BR
        goal_x,goal_y = compute_center(corners[marker3_goal][0][0],corners[marker3_goal][0][2]) # TL,BR
        #cv2.line(frame, p_tl, (int(thymio_x),int(thymio_y)), (0, 255, 0), 2) # TEST
        #cv2.line(frame, p_tl, (int(goal_x),int(goal_y)), (0, 255, 0), 2) # TEST

        thymio_theta = compute_angle(corners[marker2_thymio][0][0],corners[marker2_thymio][0][3]) # TL,BL
        cv2.putText(frame, str(thymio_theta), (thymio_x,thymio_y),cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 0), 2)
        # print(f"x={thymio_x:.1f}, y={thymio_y:.1f}") 

    # ------------------- POLYGON VERTICES DETECTION -----------------
    polygons, red_mask_roi = detect_red_polygons_in_map(frame, p_tl, p_br)
    for verts in polygons:
        cv2.polylines(frame, [verts], True, (255, 0, 255), 2)
        for (x, y) in verts:
            cv2.circle(frame, (int(x), int(y)), 4, (0, 0, 255), -1)
    # ---------------------------------------------------------------

    # Show the frame
    cv2.imshow("Aruco Feed", frame)

    # Quit with q 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
cap.release()
cv2.destroyAllWindows()
